# Fine-tuning your own model

You can do you own fine-tuning of you model from HF on our processed Poker dataset

In [ ]:
from huggingface_hub import login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from transformers import DefaultDataCollator

#login("hf_token") # Replace "your_token_here" with your actual token
device = 0 if torch.cuda.is_available() else -1

c:\Users\soelm\Documents\04_Code\MVA\MVA_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Choose your pre-trained model

In [2]:
MODEL_PATH = "HuggingFaceTB/SmolLM2-135M"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token # may be required
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

data_collator = DefaultDataCollator()

Load our dataset from HF

In [3]:
def tokenize_function(example):
    tokenized = tokenizer(
        example["question"],
        example["answer"],
        truncation="only_first",
        padding="max_length",  # Pad every example to the max length
        max_length=400         
    )
    # For causal LM training, set labels equal to input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

dataset = load_dataset("SoelMgd/Poker_Dataset")
tokenized_dataset = dataset.map(tokenize_function,batched=True,remove_columns=dataset["train"].column_names)

Map: 100%|██████████| 5243/5243 [00:01<00:00, 3498.38 examples/s]


Define your training arguments

In [7]:
training_args = TrainingArguments(
    output_dir="current_dir",    # Where to save model checkpoints
    eval_strategy="epoch",            
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,                       # Automatically push model to the Hugging Face Hub (requires authentication)
    save_strategy="epoch",                  # Save checkpoints every X training steps
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    #report_to=["tensorboard"],
    logging_dir='/kaggle/working/run'
)

Let's train!

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,                     # The tokenizer used during tokenization
    data_collator=data_collator,           
)

trainer.train()
trainer.save_model("./poker_model_finetuned")

## Fine tuning with LORA

If you prefer using LORA instead, you can use this code.
Define both your LORA config and training arguments.

In [ ]:
from peft import LoraConfig, get_peft_model

base_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

lora_config = LoraConfig(
    r=32,                         
    lora_alpha=128,              
    lora_dropout=0.05,         
    target_modules=["q_proj","k_proj","v_proj","o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./output_dir",
    evaluation_strategy="epoch",
    learning_rate=2e-5, 
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
    load_best_model_at_end=True,
    report_to=["tensorboard"],
    logging_dir='./logs'
)

In [ ]:
# Add LORA weights to the module
model = get_peft_model(base_model, lora_config)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()
trainer.save_model("./lora_poker_model")